In [1]:
import pandas as pd
import os
import numpy as np
import json
from haversine import haversine

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [2]:
store_list_Apr_2020=pd.read_csv("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200401-135137-445.txt",
                               dtype=str,sep="|")
store_list_Apr_2020['latitude_meas']=store_list_Apr_2020['latitude_meas'].astype(float)
store_list_Apr_2020['longitude_meas']=store_list_Apr_2020['longitude_meas'].astype(float)

store_list_Apr_2020=store_list_Apr_2020[['location_id','open_dt','latitude_meas','longitude_meas']]

In [3]:
store_list_YG=pd.read_excel("./OnboardingKit BL 0427.xlsx",dtype=str,sheet_name="List of stores")
store_list_YG=store_list_YG['Store'].tolist()

list_missing_location_stores=[x for x in store_list_YG if x not in store_list_Apr_2020['location_id'].tolist()]
len(list_missing_location_stores)

25

In [4]:
set([x for x in list_missing_location_stores if list_missing_location_stores.count(x)>1])

{'5422', '5423', '5424'}

In [5]:
TA_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
TA_zips=TA_zips.parse("view_by_store",dtype=str)
TA_zips.head(2)

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,1,multiple_1,1949,3178 LAVON DR,NaN,75040,GARLAND,TX,32.945015,-96.619194,{'DALLAS-FT. WORTH'},{'DALLAS'},"['75040', '75044', '75048', '75098']","['75089', '75042', '75043', '75041', '75088', ...",[],"['75040', '75098', '75044', '75048']","['75088', '75042', '75082', '75043', '75089', ..."
1,1,multiple_1,1255,1418 W MOORE AVE,NaN,75160,TERRELL,TX,32.738772,-96.299877,{'DALLAS-FT. WORTH'},"{'HUNT', 'KAUFMAN', 'ROCKWALL'}","['75160', '75126', '75161']","['75142', '75169', '75474', '75143', '75114', ...",[],"['75126', '75161', '75160']","['75142', '75169', '75150', '75117', '75143', ..."


In [6]:
df_temporary=TA_zips[['location_id','trans_P_zips_70_within_TA','trans_S_zips_70_within_TA','zips_in_10']]
df_zip_by_store=pd.DataFrame()

for ind,row in df_temporary.iterrows():
    location_id=row['location_id']
    P_zips=eval(row['trans_P_zips_70_within_TA'])
    S_zips=eval(row['trans_S_zips_70_within_TA'])
    zip_10=eval(row['zips_in_10'])
    

    df_P=pd.DataFrame(zip([location_id]*len(P_zips),P_zips))
    if len(df_P)>0:
        df_P.columns=['location_id','zip_cd']
        df_P['zip_type']="P"
        
    df_S=pd.DataFrame(zip([location_id]*len(S_zips),S_zips))
    if len(df_S)>0:
        df_S.columns=['location_id','zip_cd']
        df_S['zip_type']="S"
    
    df_10=pd.DataFrame(zip([location_id]*len(zip_10),zip_10))
    if len(df_10)>0:
        df_10.columns=['location_id','zip_cd']
        df_10['zip_type']="zip_10"
    
    df_zip_by_store=df_zip_by_store.append(df_P).append(df_S).append(df_10)

In [7]:
df_store_list_YG=pd.DataFrame({"location_id":store_list_YG},index=[0]*len(store_list_YG))
df_store_list_YG=df_store_list_YG.drop_duplicates()
df_store_list_YG['location_id']=df_store_list_YG['location_id'].astype(str)

In [8]:
df_store_list_YG['location_id'].apply(type).unique()

array([<class 'str'>], dtype=object)

In [9]:
df_store_list_YG_zips=pd.merge(df_store_list_YG,df_zip_by_store,on="location_id",how="left")
df_store_list_YG_zips_existing=df_store_list_YG_zips[pd.notnull(df_store_list_YG_zips['zip_type'])]
df_store_list_YG_zips_new=df_store_list_YG_zips[pd.isnull(df_store_list_YG_zips['zip_type'])]


In [10]:
df_stores_not_open_yet=pd.read_excel("./missing_lat_lng_stores_Store List Report 04.23.20.xlsx",
                                     sheet_name="missing_stores",dtype=str,skiprows=1)
df_stores_not_open_yet=df_stores_not_open_yet[['location_id','open_date','Google Coordinates']]
df_stores_not_open_yet.head(2)

,location_id,open_date,Google Coordinates
0,4710,2020-04-02 00:00:00,"37.5258742,-82.5685063"
1,4712,2020-09-03 00:00:00,"40.5684072,-122.3573712"


In [11]:
df_store_locations_1=store_list_Apr_2020[['location_id','open_dt','latitude_meas','longitude_meas']]
df_store_locations_1['store_coor']=df_store_locations_1[['latitude_meas','longitude_meas']].values.tolist()
df_store_locations_1=df_store_locations_1[['location_id','open_dt','store_coor']]

df_store_locations_2=df_stores_not_open_yet.rename(columns={"open_date":"open_dt","Google Coordinates":"store_coor"})
df_store_locations_2['open_dt']=df_store_locations_2['open_dt'].apply(lambda x: str(x)[:10])
df_store_locations_2['store_coor']=df_store_locations_2['store_coor'].apply(lambda x: eval("["+x+"]"))


df_store_locations_new=df_store_locations_1.append(df_store_locations_2)
df_store_locations_new['location_id']=df_store_locations_new['location_id'].apply(lambda x: str(int(x)))



In [12]:
del df_store_list_YG_zips_new['zip_cd']
del df_store_list_YG_zips_new['zip_type']
df_store_list_YG_zips_new=pd.merge(df_store_list_YG_zips_new,df_store_locations_new,on="location_id",how="left")

dict_locations_new=df_store_list_YG_zips_new.set_index("location_id").to_dict()['store_coor']

In [13]:
df_store_list_YG_zips_new=df_store_list_YG_zips_new.reset_index()
del df_store_list_YG_zips_new['index']


df_new_store_zips=pd.DataFrame()
for i, row in df_store_list_YG_zips_new.iterrows():
    store_num=row['location_id']
    store_coor=row['store_coor']
    list_store_zips=[]
    for zip_cd, v in zip_centers.items():
        dist=haversine(store_coor,v,unit="mi")
        if dist<=10:
            list_store_zips.append(zip_cd)
    df=pd.DataFrame({"zip_cd":list_store_zips},index=['zip_10']*len(list_store_zips))
    df=df.reset_index().rename(columns={"index":"zip_type"})
    df.insert(0,"location_id",[store_num]*len(list_store_zips))
    df_new_store_zips=df_new_store_zips.append(df)
    
df_store_list_YG_zips_new=pd.merge(df_store_list_YG_zips_new,df_new_store_zips,on="location_id",how="left")

In [14]:
df_store_list_YG_zips_new.shape,df_store_list_YG_zips_new['zip_cd'].nunique(),df_store_list_YG_zips_new['location_id'].nunique()

((1348, 5), 1226, 56)

In [15]:
store_list_Apr_2020_copy=store_list_Apr_2020.copy()
store_list_Apr_2020_copy['store_coor']=store_list_Apr_2020_copy[['latitude_meas','longitude_meas']].values.tolist()
store_list_Apr_2020_copy=store_list_Apr_2020_copy[['location_id','open_dt','store_coor']]

df_store_list_YG_zips_existing=pd.merge(df_store_list_YG_zips_existing,store_list_Apr_2020_copy,on="location_id",how="left")

In [16]:
df_all_zips_for_YG_stores=df_store_list_YG_zips_existing.append(df_store_list_YG_zips_new)
df_all_zips_for_YG_stores=df_all_zips_for_YG_stores[df_store_list_YG_zips_existing.columns.tolist()]

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [17]:
df_unique_zip=df_all_zips_for_YG_stores[['zip_cd','zip_type']].drop_duplicates()
df_unique_zip=df_unique_zip.sort_values(['zip_cd','zip_type'],ascending=[True,True])
df_unique_zip=df_unique_zip.drop_duplicates("zip_cd")

In [18]:
df_unique_zip.groupby("zip_type")['zip_cd'].count().to_frame().reset_index()

,zip_type,zip_cd
0,P,4686
1,S,7724
2,zip_10,1067


In [19]:
import zipcodes
df_unique_zip=df_unique_zip.reset_index()
del df_unique_zip['index']
df_unique_zip.insert(1,"city",np.nan)
df_unique_zip.insert(2,"state",np.nan)


In [20]:
list_missing_zip=[]

for i, row in df_unique_zip.iterrows():
    zip_cd=row['zip_cd']
    res=zipcodes.matching(zip_cd)
    
    if len(res)>=1:
    
        city=res[0]['city']
        state=res[0]['state']


        df_unique_zip.loc[i,"city"]=city
        df_unique_zip.loc[i,"state"]=state
    else:
        list_missing_zip.append(zip_cd)
    

In [21]:
list_missing_zip

['29486', '84009', '86005', '97003', '97703']

In [22]:
dict_missing_zip_city={
    "29486":"Summerville",
    "84009":"South Jordan",
    "86005":"Flagstaff",
    "97003":"Beaverton",
    "97703":"Bend"
}


dict_missing_zip_state={
    "29486":"SC",
    "84009":"UT",
    "86005":"AZ",
    "97003":"OR",
    "97703":"OR"    
                 }

In [23]:
df_unique_zip_0=df_unique_zip[pd.notnull(df_unique_zip['city'])]
df_unique_zip_1=df_unique_zip[pd.isnull(df_unique_zip['city'])]
df_unique_zip_1['city']=df_unique_zip_1['zip_cd'].apply(lambda x: dict_missing_zip_city[x])
df_unique_zip_1['state']=df_unique_zip_1['zip_cd'].apply(lambda x: dict_missing_zip_state[x])
df_unique_zip=df_unique_zip_0.append(df_unique_zip_1).sort_values("zip_cd")
df_unique_zip=df_unique_zip[df_unique_zip_0.columns.tolist()]

/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [24]:
df_unique_zip.shape,df_unique_zip['zip_cd'].nunique()

((13477, 4), 13477)

In [27]:
import datetime
writer=pd.ExcelWriter("./BL_TA_zips_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_unique_zip.to_excel(writer,"unique_zip_cd",index=False)
df_all_zips_for_YG_stores.to_excel(writer,"store_with_zips",index=False)
writer.save()